## 0. 패키지 설치
이번 챕터에서 사용할 패키지들인 pandas, scikit-learn, joblib 를 설치합니다.

In [1]:
# terminal-command
# !pip install pandas scikit-learn joblib

## 1. 학습 및 평가 데이터 선정
sklearn.datasets 에서 load_iris 함수를 통해서 iris 데이터를 불러옵니다.

In [2]:
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True, as_frame=True)

불러온 데이터를 각각 X, y 에 할당합니다.

sklearn.model_selection 의 train_test_split 함수를 이용해 데이터를 학습 및 평가 데이터로 나눕니다. 또한 추후에 재현할 수 있도록 random_seed 를 지정합니다.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=2022)

분리된 데이터에서 학습 데이터를 X_train, y_train 에 할당하고, 평가 데이터는 X_valid, y_valid 에 할당합니다.

## 2. 모델 개발 및 학습
데이터를 scaling 하기 위해 sklearn.preprocessing 의 StandardScaler 를 scaler 에 할당합니다.

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

fit 을 통해 scaler를 학습한 후 transform을 이용해 데이터를 scaling 합니다.

In [6]:
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_valid = scaler.transform(X_valid)

scaling 전 데이터와 scaling 후 데이터를 비교하면 다음과 같습니다.

In [7]:
print(X_train.values[0])
print(scaled_X_train[0])
# [4.4 3.  1.3 0.2]
# [-1.71687346 -0.1513372  -1.37527528 -1.29070478]

[4.4 3.  1.3 0.2]
[-1.71687346 -0.1513372  -1.37527528 -1.29070478]


In [8]:
from sklearn.svm import SVC

classifier = SVC()

In [9]:
classifier.fit(scaled_X_train, y_train)

SVC()

predict 함수를 통해 주어진 데이터에 대한 예측값을 얻을 수 있습니다. 이 때 SVC 를 scaling 이 된 데이터를 사용했기 때문에 scaled_X_train 과 scaled_X_valid 를 통해 예측을 해야 합니다.



In [11]:
train_pred = classifier.predict(scaled_X_train)
valid_pred = classifier.predict(scaled_X_valid)

학습 데이터와 평가 데이터에 대해서 예측을 진행하여 각각 train_pred 와 valid_pred 에 할당합니다.

이제 정확도를 계산해야 합니다. 정확도는 sklearn.metrics 에서 제공하는 accuracy_score 를 이용하여 계산해 보겠습니다.



In [12]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(y_true=y_train, y_pred=train_pred)
valid_acc = accuracy_score(y_true=y_valid, y_pred=valid_pred)

In [13]:
print("Train Accuracy :", train_acc)
print("Valid Accuracy :", valid_acc)
# Train Accuracy : 0.9833333333333333
# Valid Accuracy : 0.9666666666666667

Train Accuracy : 0.9833333333333333
Valid Accuracy : 0.9666666666666667


In [14]:
train_pred = classifier.predict(X_train)
valid_pred = classifier.predict(X_valid)

train_acc = accuracy_score(y_true=y_train, y_pred=train_pred)
valid_acc = accuracy_score(y_true=y_valid, y_pred=valid_pred)

print("Train Accuracy :", train_acc)
print("Valid Accuracy :", valid_acc)
# Train Accuracy : 0.23333333333333334
# Valid Accuracy : 0.23333333333333334

Train Accuracy : 0.23333333333333334
Valid Accuracy : 0.23333333333333334


/opt/homebrew/Caskroom/miniforge/base/envs/MLOps/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/MLOps/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


## 3. 학습된 모델 저장
scikit-learn 에서 공식적으로 권장하는 모델 저장 방법은 joblib 패키지를 이용하는 것입니다. [Scikit-Learn Model Presistence]

In [15]:
import joblib

joblib.dump(scaler, "scaler.joblib")
joblib.dump(classifier, "classifier.joblib")

['classifier.joblib']

## 4. 저장된 모델 불러오기
모델이 정상적으로 저장 되었는지 확인해보도록 하겠습니다. 우선, 학습할 때 나눈 데이터를 똑같이 재현합니다.

In [16]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=2022)

In [17]:
import joblib

scaler_load = joblib.load("scaler.joblib")
classifier_load = joblib.load("classifier.joblib")

In [18]:
scaled_X_train = scaler_load.transform(X_train)
scaled_X_valid = scaler_load.transform(X_valid)

In [19]:
load_train_pred = classifier_load.predict(scaled_X_train)
load_valid_pred = classifier_load.predict(scaled_X_valid)

In [20]:
from sklearn.metrics import accuracy_score

load_train_acc = accuracy_score(y_true=y_train, y_pred=load_train_pred)
load_valid_acc = accuracy_score(y_true=y_valid, y_pred=load_valid_pred)

In [21]:
print("Load Model Train Accuracy :", load_train_acc)
print("Load Model Valid Accuracy :", load_valid_acc)
# Train Accuracy : 0.9833333333333333
# Valid Accuracy : 0.9666666666666667

Load Model Train Accuracy : 0.9833333333333333
Load Model Valid Accuracy : 0.9666666666666667


## 5. 전체 코드
위에서 설명한 코드를 base_train.py, base_validate_save_model.py 로 나눠서 작성할 수 있습니다.

<br><br>
   
### 5.1 base_train.py
우선 학습 및 데이터 저장을 위한 코드들을 모은 base_train.py 입니다.

In [26]:
%%writefile base_train.py

# base_train.py
import joblib
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# 1. get data
X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=2022)

# 2. model development and train
scaler = StandardScaler()
classifier = SVC()

scaled_X_train = scaler.fit_transform(X_train)
scaled_X_valid = scaler.transform(X_valid)
classifier.fit(scaled_X_train, y_train)

train_pred = classifier.predict(scaled_X_train)
valid_pred = classifier.predict(scaled_X_valid)

train_acc = accuracy_score(y_true=y_train, y_pred=train_pred)
valid_acc = accuracy_score(y_true=y_valid, y_pred=valid_pred)

print("Train Accuracy :", train_acc)
print("Valid Accuracy :", valid_acc)

# 3. save model
joblib.dump(scaler, "scaler.joblib")
joblib.dump(classifier, "classifier.joblib")

Overwriting base_train.py


### 5.2 base_validate_save_model.py
다음은 저장된 모델을 검증하는 base_validate_save_model.py 입니다.

In [27]:
%%writefile base_validate_save_model.py
# base_validate_save_model.py

import joblib
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 1. reproduce data
X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=2022)

# 2. load model
scaler_load = joblib.load("scaler.joblib")
classifier_load = joblib.load("classifier.joblib")

# 3. validate
scaled_X_train = scaler_load.transform(X_train)
scaled_X_valid = scaler_load.transform(X_valid)

load_train_pred = classifier_load.predict(scaled_X_train)
load_valid_pred = classifier_load.predict(scaled_X_valid)

load_train_acc = accuracy_score(y_true=y_train, y_pred=load_train_pred)
load_valid_acc = accuracy_score(y_true=y_valid, y_pred=load_valid_pred)

print("Load Model Train Accuracy :", load_train_acc)
print("Load Model Valid Accuracy :", load_valid_acc)

Writing base_validate_save_model.py


# Model Pipeline

## 목표
- 여러 개의 모델들을 하나의 파이프라인으로 작성합니다.

## 스펙 명세서
1. 모델들의 파이프라인화
- scikit-learn 에 있는 pipeline 을 이용합니다.
2. 저장된 파이프라인 검증
- 저장된 파이프라인이 정상적으로 동작하는지 확인합니다.

해당 파트의 전체 코드는 [mlops-for-mle/part2/](https://github.com/mlops-for-mle/mlops-for-mle/tree/main/part2) 에서 확인할 수 있습니다.

In [ ]:
part2
├── Makefile
├── README.md
├── base_train.py
├── base_validate_save_model.py
├── db_train.py
├── db_validate_save_model.py
// highlight-next-line
├── pipeline_train.py
// highlight-next-line
└── pipeline_validate_save_model.py

# 1. Model Pipeline

## 1.1 Scaler & SVC

1) Base Model Development 챕터에서 예측을 위해 사용한 모델은 scaler 와 SVC 두 가지가 있었습니다. 또한 SVC 가 정상적으로 예측하기 위해서는 scaler 가 필요하다는 것도 알아 보았습니다. 즉 SVC 모델을 사용하기 위해서는 아래와 같이 사용해야 합니다.



In [29]:
scaled_X_train = scaler.transform(X_train)
train_pred = classifier.predict(scaled_X_train)

In [32]:
# model_pipeline.predict(X_train)

## 1.2 Code
직접 모델들을 파이프라인으로 작성해 보겠습니다.

sklearn.pipeline 의 Pipeline 을 통해 파이프라인을 작성할 수 있습니다. 이 때 파이프라인 안에 들어가는 값은 리스트이며 리스트 안에는 (모델 이름, 모델 객체) 가 값으로 들어갑니다.

In [34]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

model_pipeline = Pipeline([("scaler", StandardScaler()), ("svc", SVC())])

In [36]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('svc', SVC())])

In [37]:
print(model_pipeline[0].transform(X_train[:1]))
# [[-1.71687346 -0.1513372  -1.37527528 -1.29070478]]

[[-1.71687346 -0.1513372  -1.37527528 -1.29070478]]


In [38]:
from sklearn.metrics import accuracy_score

train_pred = model_pipeline.predict(X_train)
valid_pred = model_pipeline.predict(X_valid)

train_acc = accuracy_score(y_true=y_train, y_pred=train_pred)
valid_acc = accuracy_score(y_true=y_valid, y_pred=valid_pred)

print("Train Accuracy :", train_acc)
print("Valid Accuracy :", valid_acc)
# Train Accuracy : 0.9833333333333333
# Valid Accuracy : 0.9666666666666667

Train Accuracy : 0.9833333333333333
Valid Accuracy : 0.9666666666666667


In [39]:
import joblib

joblib.dump(model_pipeline, "model_pipeline.joblib")

['model_pipeline.joblib']

### 1.3 pipeline_train.py
위에서 작성한 코드를 모아서 pipeline_train.py 로 작성합니다

In [41]:
%%writefile pipeline_train.py
import joblib
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

# 1. get data
X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=2022)

# 2. model development and train
model_pipeline = Pipeline([("scaler", StandardScaler()), ("svc", SVC())])
model_pipeline.fit(X_train, y_train)

train_pred = model_pipeline.predict(X_train)
valid_pred = model_pipeline.predict(X_valid)

train_acc = accuracy_score(y_true=y_train, y_pred=train_pred)
valid_acc = accuracy_score(y_true=y_valid, y_pred=valid_pred)

print("Train Accuracy :", train_acc)
print("Valid Accuracy :", valid_acc)

# 3. save model
joblib.dump(model_pipeline, "model_pipeline.joblib")

Writing pipeline_train.py


## 2. pipeline_validate_save_model.py

저장된 파이프라인이 정상적으로 동작하는 지 검증하기 위해 1) Base Model Development 챕터에서 작성한 base_validate_save_model.py 의 코드를 수정하여 pipeline_validate_save_model.py 로 작성합니다.

In [42]:
%%writefile pipeline_validate_save_model.py
import joblib
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 1. reproduce data
X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=2022)

# 2. load model
model_pipeline_load = joblib.load("model_pipeline.joblib")

# 3. validate
load_train_pred = model_pipeline_load.predict(X_train)
load_valid_pred = model_pipeline_load.predict(X_valid)

load_train_acc = accuracy_score(y_true=y_train, y_pred=load_train_pred)
load_valid_acc = accuracy_score(y_true=y_valid, y_pred=load_valid_pred)

print("Load Model Train Accuracy :", load_train_acc)
print("Load Model Valid Accuracy :", load_valid_acc)

Writing pipeline_validate_save_model.py


#  Load Model from Database

## 목표
- DB 에서 데이터를 가져오는 파이프라인을 작성합니다.

## 스펙 명세서
- 데이터 불러오기
    - `01. Database` 파트에서 생성한 DB 에서 데이터를 가져옵니다.
    - id column 을 기준으로 최신 데이터 100개를 추출하는 쿼리문을 작성합니다.
    - pandas.read_sql 함수를 이용해 데이터를 추출합니다.
- 모델 파이프라인 수정
    - `1) Base Model Development` 챕터에서 작성한 파이프라인 중 데이터를 불러오는 부분을 위에서 작성한 함수로 수정합니다.
    - 모델을 학습하고 저장합니다.
    - 저장된 모델이 정상적으로 동작하는지 확인합니다.

## 1. Load Data
### 1.1 Query
id column을 기준으로 최신 데이터 100개를 추출하는 쿼리문을 작성합니다.

    SELECT * FROM iris_data ORDER BY id DESC LIMIT 100;
    
psql 에서 해당 쿼리문을 입력하면 다음과 같이 출력됩니다.

    mydatabase=# SELECT * FROM iris_data ORDER BY id DESC LIMIT 100;
    id  |         timestamp          | sepal_length | sepal_width | petal_length | petal_width | target
    -----+----------------------------+--------------+-------------+--------------+-------------+--------
       1 | 2023-01-15 06:08:58.995035 |          6.8 |         2.8 |          4.8 |         1.4 |      1
       2 | 2023-01-15 06:09:00.033342 |          6.3 |         2.5 |            5 |         1.9 |      2
       3 | 2023-01-15 06:09:01.063739 |          6.3 |         3.3 |          4.7 |         1.6 |      1
       4 | 2023-01-15 06:09:02.098688 |          6.3 |         3.3 |          4.7 |         1.6 |      1
       5 | 2023-01-15 06:09:03.131971 |            5 |         3.2 |          1.2 |         0.2 |      0
    (...)
    

### 1.2 Pandas
pandas.read_sql 는 입력 argument 로 query 와 DB connector 를 받습니다.

PostgreSQL DB 에 연결할 수 있는 DB connector 를 생성 후 query 와 DB connector 를 이용하여 데이터를 불러옵니다. DB 에 연결하기 위한 정보는 01. Database 파트의 7) Data Generator on Docker Compose 챕터에서 DB 서버를 생성할 때 입력한 값입니다.

    import pandas as pd
    import psycopg2
    
    db_connect = psycopg2.connect(host="localhost", database="mydatabase", user="myuser", password="mypassword")
    df = pd.read_sql("SELECT * FROM iris_data ORDER BY id DESC LIMIT 100", db_connect)


- db connect
    - host : localhost
    - database : mydatabase
    - user : myuser
    - password : mypassword

In [45]:
!pip install psycopg2

In [47]:
import pandas as pd
import psycopg2

db_connect = psycopg2.connect(host="localhost", database="mydatabase", user="myuser", password="mypassword")
df = pd.read_sql("SELECT * FROM iris_data ORDER BY id DESC LIMIT 100", db_connect)

/var/folders/1b/dx8_94fs0q740wx3lfknk6y40000gn/T/ipykernel_3059/3806257435.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM iris_data ORDER BY id DESC LIMIT 100", db_connect)


In [48]:
print(df.head(5))

   id                  timestamp  sepal_length  sepal_width  petal_length  \
0  12 2024-07-30 06:02:08.390820           6.4          2.7           5.3   
1  11 2024-07-30 06:02:07.386373           5.6          2.5           3.9   
2  10 2024-07-30 06:02:06.380114           6.1          3.0           4.9   
3   9 2024-07-30 06:02:05.375092           7.7          3.0           6.1   
4   8 2024-07-30 06:02:04.369931           5.7          3.8           1.7   

   petal_width  target  
0          1.9       2  
1          1.1       1  
2          1.8       2  
3          2.3       2  
4          0.3       0  


In [49]:
%%writefile db_train.py
import pandas as pd
import psycopg2
from sklearn.model_selection import train_test_split

# 1. get data
db_connect = psycopg2.connect(host="localhost", database="mydatabase", user="myuser", password="mypassword")
df = pd.read_sql("SELECT * FROM iris_data ORDER BY id DESC LIMIT 100", db_connect)
X = df.drop(["id", "timestamp", "target"], axis="columns")
y = df["target"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=2022)


Writing db_train.py


## 2. Save Data
이어서 사용한 데이터를 저장하는 # 4. save data 부분을 추가합니다. 데이터를 저장하는 이유는 현재 DB 에 계속해서 데이터가 쌓이고 있기 때문에 매번 데이터를 불러올 때마다 데이터가 바뀝니다. 데이터가 바뀌면 모델이 정상적으로 불러왔는지 확인할 수 없기 때문에 사용한 데이터를 저장하여 평가하는 부분에서 사용합니다

In [50]:
# 4. save data
df.to_csv("data.csv", index=False)

## 3. 전체 코드
### 3.1 db_train.py


In [51]:
%%writefile db_train.py

# db_train.py
import joblib
import pandas as pd
import psycopg2
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

# 1. get data
db_connect = psycopg2.connect(host="localhost", database="mydatabase", user="myuser", password="mypassword")
df = pd.read_sql("SELECT * FROM iris_data ORDER BY id DESC LIMIT 100", db_connect)
X = df.drop(["id", "timestamp", "target"], axis="columns")
y = df["target"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=2022)

# 2. model development and train
model_pipeline = Pipeline([("scaler", StandardScaler()), ("svc", SVC())])
model_pipeline.fit(X_train, y_train)

train_pred = model_pipeline.predict(X_train)
valid_pred = model_pipeline.predict(X_valid)

train_acc = accuracy_score(y_true=y_train, y_pred=train_pred)
valid_acc = accuracy_score(y_true=y_valid, y_pred=valid_pred)

print("Train Accuracy :", train_acc)
print("Valid Accuracy :", valid_acc)

# 3. save model
joblib.dump(model_pipeline, "db_pipeline.joblib")

# 4. save data
df.to_csv("data.csv", index=False)

Overwriting db_train.py


### 3.2 validate_save_model.py
다음은 2) Model Pipeline 챕터에서 저장된 모델을 검증하는 base_validate_save_model.py 를 수정해 db_validate_save_model.py 로 저장합니다. 그리고 # 1. reproduce data 에서 저장된 데이터를 불러오도록 수정합니다.



In [53]:
%%writefile db_validate_save_model.py
# db_validate_save_model.py
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 1. reproduce data
df = pd.read_csv("data.csv")
X = df.drop(["id", "timestamp", "target"], axis="columns")
y = df["target"]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=2022)

# 2. load model
pipeline_load = joblib.load("db_pipeline.joblib")

# 3. validate
load_train_pred = pipeline_load.predict(X_train)
load_valid_pred = pipeline_load.predict(X_valid)

load_train_acc = accuracy_score(y_true=y_train, y_pred=load_train_pred)
load_valid_acc = accuracy_score(y_true=y_valid, y_pred=load_valid_pred)

print("Load Model Train Accuracy :", load_train_acc)
print("Load Model Valid Accuracy :", load_valid_acc)

Writing db_validate_save_model.py
